In [1]:
import numpy as np
import simpy

processList = list()

class AMIDOLRateReward():
    def accumulateReward(self, params):
        return(0.0)
    
    def getDelay(self, params):
        return(simpy.core.Infinity)
    
    def simpyProcess(self, env, params):
        while(True):
            try:
                if(self.isEnabled(params)):
                    yield env.timeout(self.getDelay(params))
                    self.fireEvent(params)
                else:
                    yield env.timeout(simpy.core.Infinity)
                self.reactivation(env)
            except simpy.Interrupt as i:
                continue
        print(self.getName() + " terminating.")
        
class AMIDOLEvent():
    def getName(self):
        return("GenericEvent")
    
    def getRate(self, params):
        return(1.0)
    
    def getDelay(self, params):
        delay = np.random.exponential(self.getRate(params))
        return(delay)
    
    def isEnabled(self, params):
        return(True)
    
    def fireEvent(self, params):
        return(params)
    
    def reactivation(self, env):
        global processList
        for process in processList:
            if (process != env.active_process):
                process.interrupt()
    
    def simpyProcess(self, env, params):
        while(True):
            try:
                if(self.isEnabled(params)):
                    yield env.timeout(self.getDelay(params))
                    self.fireEvent(params)
                else:
                    yield env.timeout(simpy.core.Infinity)
                self.reactivation(env)
            except simpy.Interrupt as i:
                continue
        print(self.getName() + " terminating.")

In [2]:
class AMIDOLParameters():
    def __init__(self):
        self.S_Pcinfect_S = 51999999
        self.ScinfectcI_Scinfect_IcI_Pccure_I = 1
        self.ScinfectcIccure_RcR_P = 0
        self.beta = 1.0/3.0*1.24
        self.gamma = 1.0/3.0
        
class infectEvent(AMIDOLEvent):
    def getName(self):
        return("InfectEvent")
    
    def getRate(self, v):
        rate = 1.0 / (v.beta*v.S_Pcinfect_S*v.ScinfectcI_Scinfect_IcI_Pccure_I/(v.S_Pcinfect_S+v.ScinfectcI_Scinfect_IcI_Pccure_I+v.ScinfectcIccure_RcR_P))
        return(rate)
    
    def isEnabled(self, v):
        return((v.beta*v.S_Pcinfect_S * v.ScinfectcI_Scinfect_IcI_Pccure_I) > 0.0)
    
    def fireEvent(self, v):
        v.S_Pcinfect_S -= 1.0
        v.ScinfectcI_Scinfect_IcI_Pccure_I += 1.0
        
class cureEvent(AMIDOLEvent):
    
    def getName(self):
        return("CureEvent")
    
    def getRate(self, v):
        rate = 1.0/(v.gamma*v.ScinfectcI_Scinfect_IcI_Pccure_I)
        return(rate)
    
    def isEnabled(self, v):
        return(v.gamma*v.ScinfectcI_Scinfect_IcI_Pccure_I > 0.0)
    
    def fireEvent(self, v):
        v.ScinfectcI_Scinfect_IcI_Pccure_I -= 1.0
        v.ScinfectcIccure_RcR_P += 1.0
        
params = AMIDOLParameters()
cure = cureEvent()
infect = infectEvent()

env = simpy.Environment()
cureProcess = env.process(cure.simpyProcess(env, params))
processList.append(cureProcess)
infectProcess = env.process(infect.simpyProcess(env, params))
processList.append(infectProcess)

env.run(until=90.0)